In [1]:
import numpy as np
import sys
from astropy.io import fits 
#(or import pyfits as fits)
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
path_to_allstarfile = './allStar-dr17-synspec_rev1.fits'
star_hdus = fits.open(path_to_allstarfile)

# The file has multiple HDUs and we only want the data array.
data = star_hdus[1].data
# Close the remaining HDUs
star_hdus.close()
display(data)

FITS_rec([('apStar-dr17-VESTA.fits', 'VESTA', 'apo1m.calibration.VESTA', 'apogee.apo1m.stars.calibration.VESTA', 'apogee.apo1m.synspec_fix.calibration.VESTA', 'apo1m',    1, 'calibration', '',            nan,        nan, 292.21913091, -30.60291936, 99.999, 0.   , 99.999, 0.   , 99.999, 0.   , '',  0.      , 0.        ,  0.      , 0.        ,  0.     , 0.        ,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.   , 0.   ,  0.   , 0.   ,  0, 0,  0. ,  0. , '',        nan, '',         nan,        nan,           0,     4194304,           0,     4194304, 0, 0, 'APOGEE_1MTARGETAPOGEE2_1M_TARGET', 'apo1m', '',  2, 418.9276  , 441.15744 ,    0, '', 0, '',   9.257312, 10.185515  , 0.0240309 , 5980.2134, 4.643887 , -0.20365137, 0., 0., 16.54169  , 25.09239 , 20.4308   , 0, 1, 223.00002 ,  0.        , -9.99999e+03, 9999.99, -9.99999e+03, 9999.99,                   0,        nan,       nan,         nan,       nan,          nan,       nan,     nan,     nan,     nan,       nan,      nan,        nan,    

In [3]:
# determine the integer value for the bit 23 in ASPCAPFLAG
badbits = 2**23
# select for ASPCAPFLAG bit 23 not being set and for EXTRATARG to have no bits set
gd = (np.bitwise_and(data['aspcapflag'], badbits) == 0) & (data['extratarg']==0)
# 
ind = np.where(gd)[0]

In [4]:
teff_logg_check = np.logical_and(data["TEFF"] > 0, data["LOGG"] > -10) # this checks for -9999 values
teff_logg_feh_check = np.logical_and(data["FE_H"]> -6, teff_logg_check)

indices = np.where(np.logical_and(gd, teff_logg_feh_check)) 
goodStars = data[indices] # this only the good data now
display(goodStars)

FITS_rec([('apStar-dr17-2M00000002+7417074.fits', '2M00000002+7417074', 'apo25m.120+12.2M00000002+7417074', 'apogee.apo25m.stars.120+12.2M00000002+7417074', 'apogee.apo25m.synspec_fix.120+12.2M00000002+7417074', 'apo25m', 5046, '120+12', 'none', 1.03000000e-04,  74.285408, 119.40180722,  11.76741354,  8.597, 0.039,  7.667, 0.029,  7.314, 0.018, '2MASS',       nan,        nan,        nan,        nan,       nan,        nan, nan, nan, nan, nan, nan, nan, nan, nan,  7.353, 0.02 ,  7.353, 0.02 , -1, 1,  -2.8,  2.4, 'UCAC4', 0.24235167, 'RJCE_WISE', 0.24235167, 0.434002  ,           0, 0, -2147465196, 0, 0, 0, 'APOGEE2_TWOBIN_GT_0_8,APOGEE2_WISE_DERED,APOGEE2_SHORT,APOGEE2_NORMAL_SAMPLE', 'apogee2', 'disk', 3, 827.1565  , 669.4165  , 3072, 'PERSIST_MED,PERSIST_LOW',    0, '', -51.731453 , 0.20453316, 0.02059573, 3671.8015, 1.0904876, -0.4014044 , 0., 0., 82.4185   , 17.408659, 12.790598, 0, 1,  23.955153, 30.17173   , 7., 12.2, 0.8, 9999.99,  538028216707715712, 0.28544167, 0.019475  ,   0.0

In [5]:
column_names = [
    "APSTAR_ID", 
    "TARGET_ID",
    "ASPCAP_ID", 
    "FILE", 
    "APOGEE_ID",
      "TELESCOPE",
    "LOCATION_ID", "FIELD", "ALT_ID", "RA", "DEC", "GLON", "GLAT", "J",
    "J_ERR", "H", "H_ERR", "K", "K_ERR", "SRC_H", "WASH_M", "WASH_M_ERR",
    "WASH_T2", "WASH_T2_ERR", "DDO51", "DDO51_ERR", "IRAC_3_6", "IRAC_3_6_ERR",
    "IRAC_4_5", "IRAC_4_5_ERR", "IRAC_5_8", "IRAC_5_8_ERR", "IRAC_8_0",
    "IRAC_8_0_ERR", "WISE_4_5", "WISE_4_5_ERR", "TARG_4_5", "TARG_4_5_ERR",
    "WASH_DDO51_GIANT_FLAG", "WASH_DDO51_STAR_FLAG", "TARG_PMRA", "TARG_PMDEC",
    "TARG_PMSRC", "AK_TARG", "AK_TARG_METHOD", "AK_WISE", "SFD_EBV",
    "APOGEE_TARGET1", "APOGEE_TARGET2", "APOGEE2_TARGET1", "APOGEE2_TARGET2",
    "APOGEE2_TARGET3", "APOGEE2_TARGET4", "TARGFLAGS", "SURVEY", "PROGRAMNAME",
    "NINST", "NVISITS", "COMBTYPE", "COMMISS", "SNR", "SNREV", "ALL_VISITS",
    "STARFLAG", "STARFLAGS", "ANDFLAG", "ANDFLAGS", "VHELIO_AVG", "VSCATTER",
    "VERR", "VERR_MED", "RV_TEFF", "RV_LOGG", "RV_FEH", "RV_ALPHA", "RV_CARB",
    "RV_CCFWHM", "RV_AUTOFWHM", "RV_FLAG", "N_COMPONENTS", "MEANFIB", "SIGFIB",
    "MIN_H", "MAX_H", "MIN_JK", "MAX_JK", "GAIAEDR3_SOURCE_ID",
    "GAIAEDR3_PARALLAX", "GAIAEDR3_PARALLAX_ERROR", "GAIAEDR3_PMRA",
    "GAIAEDR3_PMRA_ERROR", "GAIAEDR3_PMDEC", "GAIAEDR3_PMDEC_ERROR",
    "GAIAEDR3_PHOT_G_MEAN_MAG", "GAIAEDR3_PHOT_BP_MEAN_MAG",
    "GAIAEDR3_PHOT_RP_MEAN_MAG", "GAIAEDR3_RADIAL_VELOCITY",
    "GAIAEDR3_RADIAL_VELOCITY_ERROR", "GAIAEDR3_R_MED_GEO", "GAIAEDR3_R_LO_GEO",
    "GAIAEDR3_R_HI_GEO", "GAIAEDR3_R_MED_PHOTOGEO", "GAIAEDR3_R_LO_PHOTOGEO",
    "GAIAEDR3_R_HI_PHOTOGEO", "ASPCAP_GRID", "FPARAM_GRID", "CHI2_GRID",
    "FPARAM", "FPARAM_COV", "ASPCAP_CHI2", "PARAM", "PARAMFLAG",
    "ASPCAPFLAG", "ASPCAPFLAGS", "FRAC_BADPIX", "FRAC_LOWSNR", "FRAC_SIGSKY",
    "FELEM", "FELEM_ERR", "X_H", "X_H_ERR", "X_M", "X_M_ERR", "ELEM_CHI2",
    "ELEMFLAG", "EXTRATARG", "TEFF", "TEFF_ERR", "LOGG", "LOGG_ERR",
    "VMICRO", "VMACRO", "M_H", "M_H_ERR", "ALPHA_M", "ALPHA_M_ERR", "VSINI",
    "TEFF_SPEC", "LOGG_SPEC", "C_FE", "C_FE_ERR", "CI_FE_FLAG", "CI_FE",
    "CI_FE_ERR", "C_FE_FLAG", "N_FE", "N_FE_ERR", "N_FE_FLAG", "O_FE",
    "O_FE_ERR", "O_FE_FLAG", "NA_FE", "NA_FE_ERR", "NA_FE_FLAG", "MG_FE",
    "MG_FE_ERR", "MG_FE_FLAG", "AL_FE", "AL_FE_ERR", "AL_FE_FLAG", "SI_FE",
    "SI_FE_ERR", "SI_FE_FLAG", "P_FE", "P_FE_ERR", "P_FE_FLAG", "S_FE",
    "S_FE_ERR", "S_FE_FLAG", "K_FE", "K_FE_ERR", "K_FE_FLAG", "CA_FE",
    "CA_FE_ERR", "CA_FE_FLAG", "TI_FE", "TI_FE_ERR", "TI_FE_FLAG", "TIII_FE",
    "TIII_FE_ERR", "TIII_FE_FLAG", "V_FE", "V_FE_ERR", "V_FE_FLAG", "CR_FE",
    "CR_FE_ERR", "CR_FE_FLAG", "MN_FE", "MN_FE_ERR", "MN_FE_FLAG", "FE_H",
    "FE_H_ERR", "FE_H_FLAG", "CO_FE", "CO_FE_ERR", "CO_FE_FLAG", "NI_FE",
    "NI_FE_ERR", "NI_FE_FLAG", "CU_FE", "CU_FE_ERR", "CU_FE_FLAG", "GE_FE",
    "GE_FE_ERR", "GE_FE_FLAG", "RB_FE", "RB_FE_ERR", "RB_FE_FLAG", "YB_FE",
    "YB_FE_ERR", "YB_FE_FLAG", "ND_FE", "ND_FE_ERR", "ND_FE_FLAG", "VISITS",
    "ALL_VISIT_PK", "VISIT_PK", "APSTAR_VERSION", "ASPCAP_VERSION",
    "RESULTS_VERSION"
]

nested_columns = [
    "FPARAM_GRID",
    "CHI2_GRID", 
    "FPARAM", 
    "FPARAM_COV", 
    "FELEM", 
    "FELEM_ERR",
    "X_H",
    "X_H_ERR", 
    "X_M", 
    "X_M_ERR", 
    "ELEM_CHI2", 
    "ELEMFLAG"
    "PARAM_COV"
]

bad_columns = [
    "TARG_PMSRC",
    "NINST",
    "COMBTYPE",
    "COMMISS",
    "ALL_VISITS",
    "VERR_MED",
    "GAIAEDR3_RADIAL_VELOCITY",
    "GAIAEDR3_RADIAL_VELOCITY_ERROR",
    "GE_FE",
    "GE_FE_ERR",
    "GE_FE_FLAG",
    "RB_FE",
    "RB_FE_ERR",
    "RB_FE_FLAG",
    "ND_FE",
    "ND_FE_ERR",
    "ND_FE_FLAG",
    "VISITS",
    "ALL_VISIT_PK",
    "APSTAR_VERSION",
    "ASPCAP_VERSION",
    "RESULTS_VERSION"
]

In [ ]:
# Filter out the nested columns
filtered_column_names = [col for col in column_names if col not in nested_columns]
filtered_column_names = [col for col in filtered_column_names if col not in bad_columns]

# Create an empty DataFrame
df = pd.DataFrame()

# Keep track of skipped columns
skipped_columns = []

for column in filtered_column_names:
    try:
        # Check the byte order and change to little-endian if it's big-endian
        if goodStars[column].dtype.byteorder == '>':
            df[column] = goodStars[column].byteswap().newbyteorder()
        else:
            df[column] = goodStars[column]
    except ValueError as e:
        print(f"Skipping column {column} due to error: {e}")
        skipped_columns.append(column)

In [11]:
APOGEEdfCold = df[df['TEFF']<=4500]
len(APOGEEdfCold)

115247

In [9]:
from astropy.coordinates import SkyCoord
import astropy.units as u
#Set the df to convert to crossmatch with
apogee_data = APOGEEdfCold
# Convert the Gaia data to a SkyCoord object
# Check for any NaN or None values in RA and DEC columns
# Print data types of RA and DEC columns
print("RA column data type:", apogee_data['RA'].dtype)
print("DEC column data type:", apogee_data['DEC'].dtype)
apogee_coords = SkyCoord(ra=apogee_data['RA']*u.deg, dec=apogee_data['DEC']*u.deg)
apogee_data_simple = apogee_data[['GAIAEDR3_SOURCE_ID', 'APOGEE_ID', 'RA', 'DEC']]

# Print first 5 rows of filtered data
print(f"RA constraint: ({np.min(apogee_data_simple['RA'])}, {np.max(apogee_data_simple['RA'])})")
print(f"DEC constraint: ({np.min(apogee_data_simple['DEC'])}, {np.max(apogee_data_simple['DEC'])})")
print(apogee_data_simple.columns)

apogee_data_simple = apogee_data_simple.rename(columns={'GAIAEDR3_SOURCE_ID':'source_id','APOGEE_ID':'apogee_id','RA': 'ra', "DEC": "dec"})
print(apogee_data_simple.dtypes)

display(apogee_data_simple)
#Uploaded to IRSA, ran with cone <0.1 arcsecond and ngoodobs > 100 on DR20
print(apogee_data_simple['apogee_id'].unique().size)
print(apogee_data_simple['source_id'].unique().size)

RA column data type: float64
DEC column data type: float64
RA constraint: (0.000103, 359.986523)
DEC constraint: (-86.777, 87.502434)
Index(['GAIAEDR3_SOURCE_ID', 'APOGEE_ID', 'RA', 'DEC'], dtype='object')
source_id      int64
apogee_id     object
ra           float64
dec          float64
dtype: object


,source_id,apogee_id,ra,dec
0,538028216707715712,2M00000002+7417074,0.000103,74.285408
9,422737314373993984,2M00000317+5821383,0.013232,58.360649
14,429530195985183744,2M00000546+6152107,0.022759,61.869644
15,2772085696588395776,2M00000549+1524008,0.022905,15.400249
35,420513826985325696,2M00001493+5534446,0.062212,55.579067
...,...,...,...,...
358882,2229276283631858688,2M23595526+7511338,359.980276,75.192749
358883,2016468767318259712,2M23595538+6442561,359.980780,64.715584
358885,2016199730570963968,2M23595550+6402572,359.981260,64.049225
358887,2013020767578111360,2M23595606+6251121,359.983620,62.853386


115247
113403


In [10]:
#Create IRSA comptaible file for query
from astropy.table import Table

# Convert the dataframe to an astropy table
t = Table.from_pandas(apogee_data_simple)
t.meta['comments'] = ['Cross match of APOGEE DR 19 with ZTF DR 20ish']

# Write the table to an IPAC table file
t.write('observedAPOGEECleaned.ipac', format='ascii.ipac', overwrite = True)

In [13]:
APOGEEdfCold.to_csv('APOGEE_ColdData.csv', index=False)

In [ ]:
### Light Curve Procesing Hell